# Harmonic analysis of tide gauge data with T_Tide

**Datasets this notebook expects to find** under the below given datapaths:
- `TG_rijkswaterstaat_1992_01_01-2021_12_31.csv` (combined and cleaned individual .csv files from rijkswaterstaat using function `prepare_TG_rijkswaterstaat_data` in the coastal_data module)

Files that are **created** in this notebook:
- `ttide_from_1year.csv` and `ttide_from_5years.csv` (harmonical analysis for 1 or 5 years and prediction for the entire time period, because of 18.6 years nodal cycle restriction)

**Required python modules**
- [coastal_data](https://github.com/3enedix/coastal-sea-level/tree/main/coastal_data)
- [ttide](https://github.com/moflaher/ttide_py)

In [1]:
main_datapath_input = '/home/bene/PhD-docs/80_papers/1_dataset_combination/data/input/'
main_datapath_output = '/home/bene/PhD-docs/80_papers/1_dataset_combination/data/output/'

## Imports

In [2]:
%load_ext autoreload
%autoreload 2

import ttide as tt
import pandas as pd
from zoneinfo import ZoneInfo
import matplotlib.pyplot as plt
from coastal_data import CD_statistics, CD_helper_functions

In [3]:
# matplotlib fontsizes
SMALL_SIZE = 8
MEDIUM_SIZE = 20
BIGGER_SIZE = 25

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

## Data

In [4]:
tg_data = pd.read_csv(main_datapath_input + 'TG_rijkswaterstaat_1992_01_01-2021_12_31.csv', index_col='datetime[utc]', \
                  parse_dates=['datetime[utc]'])

## Harmonic Analysis

In [5]:
shallownames = ['2SM2', 'M4', 'M6', 'M8', 'S4', 'S6', 'MK3', 'MN4', 'MS4', 'SO1', 'MSN2', 'MO3', 'MK3', 'MK4', '2MS6', 'M8']

In [6]:
# 1 year
start = pd.Timestamp('1992-01-01', tz=ZoneInfo("UTC"))
end = pd.Timestamp('1993-01-01', tz=ZoneInfo("UTC"))
tg = tg_data[(tg_data.index >= start) & (tg_data.index <= end)]
tfit_1 = tt.t_tide(tg['ssh[cm]'].values, dt=1/6, stime=start, lat=53.420034, synth=1.0, \
            shallownames=shallownames) #, constitnames=constitnames)

/home/bene/PhD-git/73_ttide_py/ttide/t_vuf.py:51: RuntimeWarning: invalid value encountered in fmod
  v = np.fmod(np.dot(const['doodson'], astro) + const['semi'], 1)


-----------------------------------
nobs = 52705 
ngood = 52705 
record length (days) = 366.01
start time: 1992-01-01 00:00:00
rayleigh criterion = 1.0

Greenwich phase computed with nodal
                   corrections applied to amplitude
                   and phase relative to center time

x0= -6.75  xtrend= 0
var(data)= 5291.80    var(prediction)= 4719.52    var(residual)= 572.36
var(prediction)/var(data) (%) = 89.2

        tidal amplitude and phase with 95 % CI estimates
 tide      freq        amp      amp_err   pha      pha_err    snr
* SA    0.0001141     8.8638     4.317    298.45     35.77       4.2
  SSA   0.0002282     0.9224     3.417     79.58    181.75     0.073
  MSM   0.0013098     3.5037     4.087    197.37     78.22      0.73
  MM    0.0015122     2.3911     4.346    309.45    107.51       0.3
  MSF   0.0028219     1.5545     3.826    318.13    146.18      0.17
  MF    0.0030501     1.5392     3.790    226.60    154.63      0.16
  ALP1  0.0343966     0.6010     0.61

In [7]:
# 5-years
start = pd.Timestamp('1992-01-01', tz=ZoneInfo("UTC"))
end = pd.Timestamp('1996-12-31', tz=ZoneInfo("UTC"))
tg = tg_data[(tg_data.index >= start) & (tg_data.index <= end)]
tfit_5 = tt.t_tide(tg['ssh[cm]'].values, dt=1/6, stime=start, lat=53.420034, synth=1.0, \
            shallownames=shallownames) #, constitnames=constitnames)

-----------------------------------
nobs = 262945 
ngood = 262945 
record length (days) = 1826.01
start time: 1992-01-01 00:00:00
rayleigh criterion = 1.0

Greenwich phase computed with nodal
                   corrections applied to amplitude
                   and phase relative to center time

x0= -7.38  xtrend= 0
var(data)= 5519.00    var(prediction)= 4788.28    var(residual)= 730.77
var(prediction)/var(data) (%) = 86.8

        tidal amplitude and phase with 95 % CI estimates
 tide      freq        amp      amp_err   pha      pha_err    snr
* SA    0.0001141     7.7050     3.869    296.02     28.81         4
  SSA   0.0002282     1.8086     3.112    189.27    105.42      0.34
  MSM   0.0013098     0.5540     2.446    100.67    214.69     0.051
  MM    0.0015122     1.3474     2.751    342.08    165.21      0.24
  MSF   0.0028219     1.4034     2.855    334.06    125.42      0.24
  MF    0.0030501     1.8375     2.925    295.30    117.57      0.39
  ALP1  0.0343966     0.3672     0

## Tidal prediction

In [8]:
tg_time = tg_data.index.to_pydatetime()
corr_1 = tfit_1(tg_time)
corr_5 = tfit_5(tg_time)
tg_corr_1 = tg_data['ssh[cm]'] - corr_1
tg_corr_5 = tg_data['ssh[cm]'] - corr_5

## Plots

## Save

In [9]:
def save_results(tg_corr_x, fname):
    save_df = pd.DataFrame(round(tg_corr_x, 2))
    save_df = save_df.rename(columns={'ssh[cm]':'ssh_corr[cm]'})
    save_df['corr[cm]'] = tg_corr_x.round(2)
    save_df.to_csv(main_datapath_output + fname)
    return save_df

save_results(tg_corr_1, 'ttide_from_1year.csv')
save_results(tg_corr_5, 'ttide_from_5years.csv')

,ssh_corr[cm],corr[cm]
datetime[utc],,
1991-12-31 23:00:00+00:00,-18.08,-18.08
1991-12-31 23:10:00+00:00,-18.97,-18.97
1991-12-31 23:20:00+00:00,-20.71,-20.71
1991-12-31 23:30:00+00:00,-21.27,-21.27
1991-12-31 23:40:00+00:00,-22.53,-22.53
...,...,...
2022-01-01 22:10:00+00:00,-2.93,-2.93
2022-01-01 22:20:00+00:00,-2.21,-2.21
2022-01-01 22:30:00+00:00,-1.55,-1.55
